# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports

import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [3]:
# set up environment

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
openai = OpenAI()

In [4]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [5]:
# Get gpt-4o-mini to answer, with streaming

system_prompt = """
You are a tutor who explains complex technical topics very well.
You break complex concepts into simpler pieces and explain each piece building up
understanding of the more complex idea step by step.
You give clear examples that makes abstract ideas concrete and easy to visualize.
You will be provided with a question. Provide your response to the question in markdown.
"""

stream = openai.chat.completions.create(
    model=MODEL_GPT,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
      ],
    stream=True
)

response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)

Sure! Let's break down the code step by step to understand what it does and why.

### Code Breakdown

python
yield from {book.get("author") for book in books if book.get("author")}


1. **`yield from`**: 
   - This is used in Python when you want to delegate part of a generator's operations to another generator. It allows you to yield all values from an iterable (like a set, list, etc.) one by one. 

2. **Set comprehension**: 
   - The expression `{book.get("author") for book in books if book.get("author")}` is a set comprehension. A set comprehension creates a set, which is a collection that automatically avoids duplicate values.

3. **Iterating through `books`**: 
   - `for book in books`: This part of the comprehension loops through each item in `books`. Here, `books` is assumed to be a collection (like a list) of dictionaries, where each dictionary likely represents a book and contains various attributes (like title, author, etc.).

4. **Getting the author**: 
   - `book.get("author")`: This retrieves the value associated with the key `"author"` from each book dictionary. 
     - `.get()` is a method that safely accesses a dictionary's key and returns `None` if the key doesn't exist, instead of raising a KeyError.

5. **Filtering out None values**:
   - `if book.get("author")`: This condition checks if an author exists (i.e., it is not `None`) before including it in the set. If `book.get("author")` returns `None`, that book will be skipped.

### What Does the Code Do?

- Essentially, this code is creating a set of unique authors from a list of books while filtering out any books that do not have an author.
- By wrapping this in `yield from`, it allows to yield each unique author one at a time from a generator.

### Example

Let’s consider an example to make this clearer.

python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3", "author": None},
    {"title": "Book 4", "author": "Author A"},
]

def unique_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

# Using the generator
for author in unique_authors(books):
    print(author)


### Output

Author A
Author B


### Conclusion

- The `unique_authors` function processes the `books` list, extracting the `author` field from each book dictionary, filtering out any that are `None`, and then using `yield from` to yield each unique author one at a time.
- This is useful when you want to generate a list of authors efficiently without creating a whole list in memory upfront, especially when working with large datasets.

In [12]:
# Get Llama 3.2 to answer

ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
stream = ollama_via_openai.chat.completions.create(
    model=MODEL_LLAMA,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ],
    stream=True
)

response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)

**Breaking Down the Code**
=========================

The given code snippet is written in Python and utilizes a concept called generators, which produce a series of values over time. Let's break it down:


# Dictionary comprehension with a generator expression
values = {book.get("author") for book in books if book.get("author")}


This dictionary comprehension is used to create a new dictionary (`values`) containing only the `author` values from the existing list of dictionaries (`books`). However, instead of creating the entire dictionary at once with all the data, this expression uses generators.

**Why Use Generators?**
---------------------

Generators are useful when dealing with large datasets or infinite sequences because they yield values one by one. In Python, you can use the `yield from` keyword, which allows the generator to "give credit" to another generator, passing on its generated values instead of having to generate them again.

**Code Explaination**
---------------------


# This line defines a new generator expression.
yield from {book.get("author") for book in books if book.get("author")}


Here's how it works:

- `{book.get("author") for book in books if book.get("author")}` creates an iterator, which generates values on-the-fly.

The `yield from` keyword then comes into play. This statement combines the two generators together. It takes all values produced by the sub-generator (`{book.get("author") for book in books if book.get("author")}`) and yields them as part of its own sequence.

**Equivalent Non-Generator Code**
---------------------------------

Here's how you might write this using a function to generate the author names directly:


def get_authors(books):
    authors = []
    for book in books:
        if book.get("author"):
            authors.append(book.get("author"))
    yield from authors

authors = list(get_authors(books))


However, as can be seen, the original code produces the generators in a single line of code.

**Example Usage**
-----------------


# List of books with their authors.
books = [
    {"title": "Harry Potter", "author": "JK Rowling"},
    {"title": "1984", "author": None},
    {"title": "The Lord of the Rings", "author": "JRR Tolkien"}
]

authors = {book.get("author") for book in books if book.get("author")}
for author in authors:
    print(author)
